# TFLearn, Computer Vision, 2. Convolutional Network (MNIST)

In [1]:
# -*- coding: utf-8 -*-

""" Convolutional Neural Network for MNIST dataset classification task.

References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.

Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/

"""

from __future__ import division, print_function, absolute_import

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [7]:
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
print("---")
print("X shape:", X.shape)
X = X.reshape([-1, 28, 28, 1])
print("X re-shaped:", X.shape)
print("testX shape:", testX.shape)
testX = testX.reshape([-1, 28, 28, 1])
print("testX re-shaped:", testX.shape)
print("Y shape:", Y.shape)
print("testY shape:", testY.shape)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
---
X shape: (55000, 784)
X re-shaped: (55000, 28, 28, 1)
testX shape: (10000, 784)
testX re-shaped: (10000, 28, 28, 1)
Y shape: (55000, 10)
testY shape: (10000, 10)


In [5]:
for optimizer in ['SGD', 'Adam']:
    for learning_rate in [0.1, 0.01, 0.001]:
        for n_epoch in [1]:
            
            run_id = optimizer + ', ' + str(learning_rate) + ", n" + str(n_epoch)
            
            tf.reset_default_graph()

            network = input_data(shape=[None, 28, 28, 1], name='input')
            
            network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
            network = max_pool_2d(network, 2)
            network = local_response_normalization(network)

            network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
            network = max_pool_2d(network, 2)
            network = local_response_normalization(network)

            network = fully_connected(network, 128, activation='tanh')
            network = dropout(network, 0.8)

            network = fully_connected(network, 256, activation='tanh')
            network = dropout(network, 0.8)

            network = fully_connected(network, 10, activation='softmax')

            network = regression(
                network, 
                optimizer=optimizer, 
                learning_rate=learning_rate, 
                loss='categorical_crossentropy', 
                name='target'
            )
            
            model = tflearn.DNN(network, tensorboard_verbose=3)
            model.fit(
                {'input': X}, {'target': Y},
                n_epoch=n_epoch,
                validation_set=({'input': testX}, {'target': testY}),
                snapshot_step=100, 
                show_metric=True, 
                run_id=run_id
            )            

Training Step: 859  | total loss: 0.07538 | time: 42.243s
| Adam | epoch: 001 | loss: 0.07538 - acc: 0.9781 -- iter: 54976/55000
Training Step: 860  | total loss: 0.07538 | time: 43.385s
| Adam | epoch: 001 | loss: 0.07538 - acc: 0.9772 | val_loss: 0.05711 - val_acc: 0.9813 -- iter: 55000/55000
--
